In [1]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject

# 1) Choose ACTIVE DATA (training or scoring)

## 1A) Empty contructor (R&D scenario: Run notebooks `or` Production scenario: Define BATCH-pipeline))
- Reads from settings folder for `lake_settings`,  for static model definition [`label, datasets to use`]
- Read from DataLake `train/active` folder for [`train_in_folder_date`] and `inference/active`for [`scoring_folder_date,inference_model_version,train_in_folder_date`]
    - Note: If no json in datalake, it falls back on LOCAL `setting/active` code folder (GIT)
- Note: `p.describe()` will show the LOCAL code folder...it is not until you run `p.init()` or `p.connect_to_lake()` we know IF lake `active` files exists, and overrides

In [32]:
p = ESMLProject() # Great when "working with 1 model at the time" (R&D or PRODUCTION for pipeline-creation)
p.inference_mode = True
#p.active_model = 12 #  settings/project_specific/model/lake_settings.json
p.describe()

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL
Inference version: 2

 - ds01_diabetes
projects/project002/11_diabetes_model_reg/inference/2/ds01_diabetes/in/dev/2021/06/11/
projects/project002/11_diabetes_model_reg/inference/2/ds01_diabetes/out/bronze/dev/
projects/project002/11_diabetes_model_reg/inference/2/ds01_diabetes/out/silver/dev/

 - ds02_other
projects/project002/11_diabetes_model_reg/inference/2/ds02_other/in/dev/2021/06/11/
projects/project002/11_diabetes_model_reg/inference/2/ds02_other/out/bronze/dev/
projects/project002/11_diabetes_model_reg/inference/2/ds02_other/out/silver/dev/
 

Training GOLD (p.GoldPath)
projects/project002/11_diabetes_model_reg/inference/2/gold/dev/
 

[A) USAGE]: to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path()
A)INFERENCE ONLINE: GOLD to score (example if realtime - today)
projects/project002/11_diabetes_model_reg/inference/2/gold/dev/2021_10_05/979585b4f1cc4257b62732bf6cd9b3e0/


## Lets see if DATLAKE contains `active` folders with files, that overrides our local settings

In [33]:
p.ws = p.get_workspace_from_config()
data_store = p.connect_to_lake()

Using GEN2 as Datastore
Searching for setting in ESML datalake...
ESML in-folder settings override = TRUE 
 - Found settings in the ESML AutoLake  [active_in_folder.json,active_scoring_in_folder.json], to override ArgParse/GIT config with.
 - TRAIN in date:  2021/01/01
 - INFERENCE in date: 2021/06/08 and ModelVersion to score with: 1 (0=latest)


In [37]:
print("AFTER we connected to the DATALAKE, if ACTIVE folder with files exists in datalake, parameters will be overriden by the DATLAKE parameters")
print("This is a convenient way to have SETTINGS per model, in the datalake, for Azure Data factory or other service to overwrite...not having to know about ESML SDK. No dependency to Python")
print("")
print("Model version:", p.inferenceModelVersion)
print("IN DATA - date_folder:", p.InDateFolder)

AFTER we connected to the DATALAKE, if ACTIVE folder with files exists in datalake, parameters will be overriden by the DATLAKE parameters
This is a convenient way to have SETTINGS per model, in the datalake, for Azure Data factory or other service to overwrite...not having to know about ESML SDK. No dependency to Python

Model version: 1
IN DATA - date_folder: 2021/01/01


## 1B) Contructor with parameters (MLOps scenario: SCORING or RETRAINING pipeline)
- Reads from settings folder for `lake_settings`, for static model definition `[label, datasets to use]`
- Overrides LOCAL `active` folder. `AND override` even if .json exists in DATALAKE `active` folders, or in local code `active` folder 
    - This overwrites the local .json files (GIT), AND also prohibits DATALAKE files to override. 

### Note: `p.describe()` will show the LOCAL code folder...it is not until you run `p.init()` or `p.connect_to_lake()` we know IF lake `active` files exists, and see that lake-override is prohibited
- This will show in output logs: `ESML in-folder settings override = FALSE.`
- Contructor wins over all external injection of settings

### 1b) USUAL SCENARIO: MLOps scenario or Scoring scenario
- `MLOps`: Parameters Azure Devops (or Azure Datafactory) passes for continous `RETRAINING (CI/CD)`, to initate the ESMLProject constructor
- `Scoring`: Parameters Azure Datafactory passes to the Azure ML Pipline for continous `SCORING`

In [39]:
param_esml_env = "dev" 
param_inference_model_version = "2" # DATALAKE(my_model/inference/active) | settings/project_specific/active/active_scoring_in_folder.json
param_scoring_folder_date = "2021-06-11 00:00:01.243860" # DATALAKE(my_model/inference/active) | settings/project_specific/active/active_scoring_in_folder.json

param_train_in_folder_date = "2021-01-11 00:00:01.243860" # DATALAKE(my_model/train/active) | settings/project_specific/active/active_in_folder.json

In [40]:
p = ESMLProject(param_esml_env,param_inference_model_version,param_scoring_folder_date,param_train_in_folder_date)
p.inference_mode = True
p.active_model = 12

#p.ws = p.get_workspace_from_config() #2) Ensure workspace is set
p.describe()

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL
json_date_in_folder {'dev_in_folder_date': '2021-01-11 00:00:01.243860', 'test_in_folder_date': '2020-01-01 15:35:01.243860', 'prod_in_folder_date': '2020-01-01 15:35:01.243860'}
json_date_scoring_folder {'dev_scoring_folder_date': '2021-06-11 00:00:01.243860', 'test_scoring_folder_date': '2020-01-01 15:35:01.243860', 'prod_scoring_folder_date': '2020-01-01 15:35:01.243860', 'dev_inference_model_version': '2', 'test_inference_model_version': '0', 'prod_inference_model_version': '0'}
Inference version: 2

 - ds01_vw
projects/project002/12_car_price_regression/inference/2/ds01_vw/in/dev/2021/06/11/
projects/project002/12_car_price_regression/inference/2/ds01_vw/out/bronze/dev/
projects/project002/12_car_price_regression/inference/2/ds01_vw/out/silver/dev/

 - ds02_audi
projects/project002/12_car_price_regression/inference/2/ds02_audi/in/dev/2021/06/11/
projects/project002/12_car_price_regression/inferen

## Lets see if DATLAKE contains `active` folders with files, that overrides our local settings

In [41]:
p.ws = p.get_workspace_from_config()
data_store = p.connect_to_lake()

Using GEN2 as Datastore
Searching for setting in ESML datalake...
ESML in-folder settings override = FALSE. [active_in_folder.json,active_scoring_in_folder.json] not found. 
 - Using [active_in_folder.json,active_scoring_in_folder.json] from ArgParse or GIT. No override from datalake settings


In [42]:
print("Even AFTER we connected to the DATALAKE, the Constructor parameters persists, as can be seen")
print("Model version:", p.inferenceModelVersion)
print("IN DATA - date_folder:", p.InDateFolder)

Even AFTER we connected to the DATALAKE, the Constructor parameters persists, as can be seen
Model version: 2
IN DATA - date_folder: 2021/01/11


# 2) Choose ACTIVE MODEL & dataset filters: `LAKE_SETTINGS`
- [`label, dataset_array`,`model_name`,`model_alias`] is sort of more `static` than the data that feeds into the `dataset` folders
- How you initate the ESMLProject and choose ACTIVE DATA is one thing. Another thing is to selecte ACTIVE MODEL, and `dataset filters`

## 2A) `STATIC` settings GIT: Model settings from (lake_settings.json) default
- `MLOps purpose`: Azure Devops to use a GIT-branch for your model per environment, such as `M01_dev`, `M01_test`, `M01_prod`,

In [45]:
p = ESMLProject() # Great when "working with 1 model at the time" (R&D or PRODUCTION for pipeline-creation)
p.inference_mode = True
#p.active_model = 12 # Reads from: settings/project_specific/model/lake_settings.json
p.describe()

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL
Inference version: 2

 - ds01_diabetes
projects/project002/11_diabetes_model_reg/inference/2/ds01_diabetes/in/dev/2021/06/11/
projects/project002/11_diabetes_model_reg/inference/2/ds01_diabetes/out/bronze/dev/
projects/project002/11_diabetes_model_reg/inference/2/ds01_diabetes/out/silver/dev/

 - ds02_other
projects/project002/11_diabetes_model_reg/inference/2/ds02_other/in/dev/2021/06/11/
projects/project002/11_diabetes_model_reg/inference/2/ds02_other/out/bronze/dev/
projects/project002/11_diabetes_model_reg/inference/2/ds02_other/out/silver/dev/
 

Training GOLD (p.GoldPath)
projects/project002/11_diabetes_model_reg/inference/2/gold/dev/
 

[A) USAGE]: to_score_folder, scored_folder, date_folder = p.get_gold_scored_unique_path()
A)INFERENCE ONLINE: GOLD to score (example if realtime - today)
projects/project002/11_diabetes_model_reg/inference/2/gold/dev/2021_10_06/e4b5047392554cbf8c9fb7782f854e1d/


## 2A) `DYNAMIC` settings GIT: Model settings from `p.active_model = 2`
- `R&D Purpose`: To have each notebook `override` the model configs [`label, dataset_array`,`model_name`,`model_alias`]
- `Production Purpose`: To have a notebook that creates a BATCH SCORING Pipeline for production, to `override` each model
    - Example: If you have a multi-output regression use case, you can have a `loop` to just switch the 

In [47]:
p = ESMLProject() # Great when "working with 1 model at the time" (R&D or PRODUCTION for pipeline-creation)
p.inference_mode = True
p.active_model = 10 # Overrides: settings/project_specific/model/lake_settings.json
p.describe()

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL
Inference version: 2

 - ds01_titanic
projects/project002/10_titanic_model_clas/inference/2/ds01_titanic/in/dev/2021/06/11/
projects/project002/10_titanic_model_clas/inference/2/ds01_titanic/out/bronze/dev/
projects/project002/10_titanic_model_clas/inference/2/ds01_titanic/out/silver/dev/

 - ds02_haircolor
projects/project002/10_titanic_model_clas/inference/2/ds02_haircolor/in/dev/2021/06/11/
projects/project002/10_titanic_model_clas/inference/2/ds02_haircolor/out/bronze/dev/
projects/project002/10_titanic_model_clas/inference/2/ds02_haircolor/out/silver/dev/

 - ds03_housing
projects/project002/10_titanic_model_clas/inference/2/ds03_housing/in/dev/2021/06/11/
projects/project002/10_titanic_model_clas/inference/2/ds03_housing/out/bronze/dev/
projects/project002/10_titanic_model_clas/inference/2/ds03_housing/out/silver/dev/

 - ds04_lightsaber
projects/project002/10_titanic_model_clas/inference/2/ds04